# Example: JUSTICE-MATTER - IAM Simulation Run

In [1]:
import pandas as pd
import numpy as np

from src.util.data_loader import DataLoader
from src.util.enumerations import *
from src.util.model_time import TimeHorizon

## Setting Recycling Rate

JUSTICE-MATTER allows for a third policy lever: Recyling Rate. To run the simulation version of the model, we need to feed the policy levers with values we want to test. The ranges of the policy levers are from 0 to 1.0 corresponding to 0% to 100%

In [3]:
import h5py

file_path = "data/input/recycling_rates_arrays"
# In use stock initial values
# Open the HDF5 file and read the dataset
with h5py.File(file_path, 'r') as f:
    recycling_rate_init = f['recycling_rate_2015'][:]

# Display the initial recycling rates
print(recycling_rate_init)

PermissionError: [Errno 13] Unable to synchronously open file (unable to open file: name = 'data/input/recycling_rates_arrays', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)

## Initialize model, select Scenario

In [ ]:
# Get list of Scenarios from Enum

for idx, scenarios in enumerate(list(Scenario.__members__.keys())):
    print(idx, scenarios)

In [ ]:
from src.model import JUSTICE
from src.util.enumerations import Economy, DamageFunction, Abatement, WelfareFunction, EconomySubModules

# Instantiate the model
scenarios = 3
model = JUSTICE(
    scenario=scenarios,
    economy_type=Economy.NEOCLASSICAL,
    damage_function_type=DamageFunction.KALKUHL,
    abatement_type=Abatement.ENERDATA,
    social_welfare_function=WelfareFunction.UTILITARIAN,
    matter=EconomySubModules.MATTER,  # Specify the economy submodule as MATTER
)

## Stepwise Run & Evaluate

In [ ]:
# Run the model
time_horizon = TimeHorizon(start_year=2015, end_year=2300, data_timestep=5, timestep=1)
for timestep in range(len(time_horizon.model_time_horizon)):
    model.stepwise_run(
        emission_control_rate=emissions_control_rate[:, timestep], 
        timestep=timestep, 
        endogenous_savings_rate=True, 
        recycling_rate=recycling_rate
    )
    datasets = model.stepwise_evaluate(timestep=timestep)

Extracting the data from the outcome dictionary to the different variables

In [ ]:
#Load the data
depletion = datasets['depletion_ratio']
emissions_avoided =datasets['emissions_avoided']
dmc = datasets['material_consumption']
recycled = datasets['recycled_material']
extracted = datasets['extracted_matter']
waste = datasets['waste']
discard = datasets['discarded_material']
reserves = datasets['material_reserves']
resources = datasets['material_resources']
r_costs= datasets['recycling_cost']

gross_output = datasets['gross_economic_output'] 
net_output = datasets['net_economic_output']
consumption = datasets['consumption'] #(57, 286, 1001)
cpc = datasets['consumption_per_capita'] #(57, 286, 1001)
emis_control = datasets['emissions'] #(57, 286, 1001)
reg_temp = datasets['regional_temperature']
temp = datasets['global_temperature'] # (286, 1001)
damages = datasets['economic_damage'] #(57, 286, 1001)
abatecost = datasets['abatement_cost'] #(57, 286, 1001)
disentangled_utility = datasets['disentangled_utility'] #(57, 286, 1001)
welfare = datasets['welfare'] # (1001,)
welfare_temporal = datasets['welfare_temporal']  # (286, 1001)
welfare_regional = datasets['welfare_regional'] # (57, 1001)
welfare_regional_temporal = datasets['welfare_regional_temporal'] # (57, 286, 1001)


## Visualize

We can visualize the timeseries of the different outcomes that we extracted in the previous step

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create list of all the data arrays
data_list = [dmc, discard, recycled,
 emissions_avoided, waste, extracted, emis_control, reserves,
 depletion, resources, r_costs
 ]
titles = ['DMC (Gt)', 'Discarded Material (Gt)',
    'Recycled Material (Gt)', 'Emis Avoided (Gt)', 'Waste (Gt)',
    'Extracted Matter (Gt)', 'Emissions (Gt)','Reserves (Gt)', 'Depletion Ratio',
    'Material Resources (Gt)', 'Recycling Costs'
]

region_index = 54
# Number of steps to plot
num_steps = 86

# Slice the time horizon to the first 86 steps
time_horizon_sliced = time_horizon.model_time_horizon[:num_steps]

# Create a figure with 4 rows and 3 columns
fig, axs = plt.subplots(4, 3, figsize=(15, 15))

# Flatten the axs array to iterate over it
axs = axs.flatten()

# Iterate over the data arrays and plot them
for i, loaded_data in enumerate(data_list):
    # Select the region based on region_index
    region_data = loaded_data[region_index, :, :]
    
    # Limit to the first 86 steps
    region_data_sliced = region_data[:num_steps, :]
    
    # Create a line plot for each scenario
    for j in range(region_data_sliced.shape[1]):
        sns.lineplot(x=time_horizon_sliced, y=region_data_sliced[:, j], ax=axs[i])
    
    # Set the title and axis labels
    axs[i].set_title(titles[i])
    axs[i].set_xlabel('Year')
    axs[i].set_ylabel('Value')

# Remove the unused subplots
for i in range(len(data_list), len(axs)):
    fig.delaxes(axs[i])

# Adjust the layout and spacing
fig.tight_layout()
plt.subplots_adjust(hspace=0.5)

plt.show()
